# DS105A: Data for Data Science: Summative 01
## Part 1
1. Scrap the list of events from the [Civica Schedule Page](https://socialdatascience.network/#schedule)

In [322]:
import requests
import pandas as pd

from pprint import pprint
from scrapy import Selector

In [323]:
toScrape = 'https://socialdatascience.network/#schedule'
response = requests.get(toScrape)

In [324]:
sel = Selector(text=response.text)

2. Produce a dataframe called 'df_schedule' with the columns 'title', 'speaker', 'date', and 'link'

In [325]:
cards = sel.css("div.card-body")

# collecting all the event titles
titles = cards.css('h6 ::text').getall()

# collecting all the event speakers
datesSpeakers = cards.css("p ::text").getall()
speakers = datesSpeakers[::2]
for i in range(len(speakers)):
    if "Speaker: " not in speakers[i]:
        speakers[i] = ""
    else:
        speakers[i] = speakers[i][9::]

# collecting all the event dates and converting them to the date data type
dates = datesSpeakers[1::2]
for i in range(len(dates)):
    dates[i] = dates[i][6:]
    dates[i] = pd.to_datetime(dates[i])

# collecting all the event links
links = cards.css('a ::attr(href)').getall()
for i in range(len(links)):
    links[i] = 'https://socialdatascience.network/' + links[i]

links = links[::2]
# better way to remove duplicates??


# producing a dataframe called df_schedule
df_schedule = pd.DataFrame({
    'Title': titles,
    'Date': dates,
    'Speakers': speakers,
    'Link': links,
})

3. Save it to a CSV file named 'schedule.csv' in the data folder of your repository.

In [326]:
df_schedule.to_csv('./data/schedule.csv')

## Part 2

In [396]:
eventLinks = []
description = []
allSpeakers = []
talkSubtitles = []

for i in range(len(df_schedule)):
    newScraping = links[i]
    response = requests.get(newScraping)
    sel = Selector(text=response.text)
    agenda = sel.css("div.col-md-10")
    
    ls_h4 = agenda.css("h4").getall()
    ls_h4_text = agenda.css("h4 ::text").getall()

    # collecting the list of the speakers
    speakerNames = agenda.css("span ::text").getall()
    x = 0
    for i in range(len(ls_h4)):
        if "<span>" in ls_h4[i]:
           allSpeakers.append(speakerNames[x])
           x += 1
        else: 
            allSpeakers.append("")
            
    # collecting the list of the event titles
    for i in range(len(ls_h4_text)):
        for i in range(len(speakerNames)):
            if speakerNames[i] in ls_h4_text:
                ls_h4_text.remove(speakerNames[i])
    talkSubtitles.extend(ls_h4_text)

    # collecting the list of links
    for i in range(len(ls_h4_text)):
        eventLinks.append(newScraping)

    # collecting the descriptions of each talk
    ls_p = agenda.css("p ::text").getall()
    ls_bold = agenda.css("b ::text").getall()

    for i in range(len(ls_p)):
        for j in range(len(ls_bold)):
            if ls_p[i] == ls_bold[j]:
                ls_p[i] = ls_p[i] + ls_p[i + 1]
            description.append(ls_p)
    

df_agenda = pd.DataFrame({
    'Event Link': eventLinks,
    'Talk Title': talkSubtitles,
    'Talk Speaker': allSpeakers,
    # 'Talk Description': description
})


df_agenda.to_csv('./data/agenda.csv')

187


## Additional Insights

While coding, I realised that for certain CIVICA events, the 'Agenda' table is not consistent across the entire website. Therefore, in order to successfully scrape all the events in the desired way (i.e. to add information about the speakers, event titles, event links and descriptions to the agenda.csv file), I must account for this.

This discrepancy is visible on the website: https://socialdatascience.network/polarisation/2023.html 

In [ ]:
# code for this here

While attempting to gain all the descriptions, I noticed that some of the descriptions have further information about speakers, but these are not shown within "p" tags, but inside "li" ones. To fully scrape the descriptions of each talk, this must be accounted for.

This is visible on the website: https://socialdatascience.network/launch.html 

In [367]:
containsList = False
if "<li>" in agenda:
    containsList += 1

print(containsList)

False
